In [4]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import geopandas as gpd
from trino.dbapi import connect
# import folium
import param
import os

import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
from holoviews import opts

from teehr.querying.utils import df_to_gdf

from typing import Union
from datetime import datetime, timedelta

import utils

In [14]:
# conn = utils.get_trino_connection()
# query = """
# SELECT * FROM iceberg.teehr.locations WHERE id = 'usgs-09032050'
# """
# df = pd.read_sql(query, conn)
# df_to_gdf(df)

In [17]:
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

In [18]:
pn.extension('tabulator', design="material", sizing_mode="stretch_width")

In [ ]:
class DashboardState(param.Parameterized):
    locations_gdf = param.ClassSelector(class_=gpd.GeoDataFrame, default=None)
    metrics_gdf = param.ClassSelector(class_=gpd.GeoDataFrame, default=None)
    current_map_gdf = param.ClassSelector(class_=gpd.GeoDataFrame, default=None)  # Track filtered data
    selected_index = param.Integer(default=0)
    location_id = param.String(default="")
    configuration_names = param.List(default=[])
    variable_names = param.List(default=[])
    metrics = param.List(default=[])
    timeseries_loading = param.Boolean()
    # selected_configuration_name = param.String(default=None)
    # selected_metric = param.String(default=None)

    @param.depends('selected_index', watch=True)
    def update_location_id(self):
        # Use the current filtered map data instead of the full metrics_gdf
        if hasattr(self, 'current_map_gdf') and self.current_map_gdf is not None and not self.current_map_gdf.empty:
            try:
                self.location_id = str(self.current_map_gdf.iloc[self.selected_index]['primary_location_id'])
                # print("Setting location_id to:", self.location_id)
            except Exception as e:
                self.location_id = ""
                print("Error occurred while setting location_id.  Setting to empty string.", e)
        elif hasattr(self, 'metrics_gdf') and self.metrics_gdf is not None:
            try:
                self.location_id = str(self.metrics_gdf.iloc[self.selected_index]['primary_location_id'])
                # print("Setting location_id to:", self.location_id)
            except Exception as e:
                self.location_id = ""
                print("Error occurred while setting location_id.  Setting to empty string.", e)
        else:
            self.location_id = ""
            print("Locations GeoDataFrame not found. Setting location_id to empty string.")

    @param.depends('metrics_gdf', watch=True)
    def update_metrics(self):
        if hasattr(self, 'metrics_gdf') and self.metrics_gdf is not None:
            self.configuration_names = self.metrics_gdf['configuration_name'].unique().tolist()
            self.variable_names = self.metrics_gdf['variable_name'].unique().tolist()
            self.metrics = self.metrics_gdf.columns.difference(['geometry', 'primary_location_id', 'name', 'configuration_name', 'variable_name']).tolist()
        else:
            self.configuration_names = []
            self.variable_names = []
            self.metrics = []

state = DashboardState()

In [5]:
# Load Timeseries Button - defined early so it can be referenced in callbacks
load_timeseries_button = pn.widgets.Button(
    name="Load Timeseries", 
    button_type="primary",
    width=150,
    margin=(10, 5)
)

In [6]:
# Selected location callback - only sets the location, doesn't load timeseries
def selection_callback(index):
    if len(index) > 1:
        print("Multiple selection detected, using the first selected index.")
    
    # Store previous location to detect changes
    previous_location = state.location_id
    state.selected_index = index[0]
    
    # Clear timeseries loading trigger when location changes
    if state.location_id != previous_location:
        # Reset the button click counter to clear the timeseries display
        load_timeseries_button.clicks = 0
        print(f"Location changed to: {state.location_id}. Timeseries cleared.")
    else:
        print(f"Location selected: {state.location_id}")
        
    # Note: We no longer set timeseries_loading = True here
    # User must click the button to load timeseries

In [7]:
def get_locations():
    """Fetches location data from the Trino database and returns it as a GeoDataFrame."""
    conn = utils.get_trino_connection()
    query = """
    SELECT * FROM iceberg.teehr.locations
    WHERE id LIKE 'usgs-%'
    """
    df = pd.read_sql(query, conn)
    gdf = df_to_gdf(df)
    return gdf

locations_gdf = get_locations()
state.locations_gdf = locations_gdf

In [8]:
# @pn.cache
def get_metrics_by_location():
    """Fetches simulation metrics by location from the Trino database and returns it as a GeoDataFrame."""
    conn = utils.get_trino_connection()
    sql = "SELECT * from iceberg.teehr.sim_metrics_by_location"
    df = pd.read_sql(sql, conn)
    gdf = df_to_gdf(df)
    
    return gdf

metrics_gdf = get_metrics_by_location()
state.metrics_gdf = metrics_gdf


In [9]:
def get_timeseries_plot(
    location_id: str, 
    load_trigger: int = 0,  # Button click counter to trigger loading
    start_date: Union[str, datetime, pd.Timestamp, None] = None, 
    end_date: Union[str, datetime, pd.Timestamp, None] = None
):
    if not location_id:
        return pn.pane.Markdown("## Select a location to view its time series data.")
    
    if load_trigger == 0:
        return pn.pane.Markdown(f"## Location {location_id} selected. Click 'Load Timeseries' to view data.")

    # Set loading state
    state.timeseries_loading = True
    
    try:
        secondary_df = utils.get_secondary_timeseries(
            location_id, 
            configuration_names=state.configuration_names,
            start_date=start_date,
            end_date=end_date
        )

        primary_df = utils.get_primary_timeseries(
            location_id,
            start_date=start_date,
            end_date=end_date
        )
        
        primary_plot = primary_df.hvplot.line(
            x='value_time', 
            y='value', 
            by='configuration_name',
            label='Primary (Observed)'
        )
        
        secondary_plot = secondary_df.hvplot.line(
            x='value_time', 
            y='value', 
            by='configuration_name',
            label='Secondary (Simulated)'
        )

        return (primary_plot * secondary_plot).opts(
            title=f"Observed and Simulated Timeseries at {location_id}"
        )
    
    except Exception as e:
        return pn.pane.Markdown(f"## Error loading timeseries data: {str(e)}")
    
    finally:
        # Clear loading state
        state.timeseries_loading = False

# We'll create the bound function later after defining the button

In [ ]:
def get_locations_map(metrics_gdf: gpd.GeoDataFrame = None, configuration_name: str = None, metric: str = None, variable_name: str = None):
    
    if configuration_name:
        gdf = metrics_gdf[metrics_gdf['configuration_name'] == configuration_name]
    else:
        gdf = metrics_gdf[metrics_gdf['configuration_name'] == state.configuration_names[0]]

    if variable_name:
        gdf = gdf[gdf['variable_name'] == variable_name]
    else:
        gdf = gdf[gdf['variable_name'] == state.variable_names[0]]
        
    if not metric:
        metric = state.metrics[0]
    
    # Store the filtered dataframe for selection mapping
    state.current_map_gdf = gdf.copy()
    
    tiles = gv.tile_sources.OSM
    gdf_prj = gdf.to_crs("EPSG:3857")
    
    points = gv.Points(
        gdf_prj,
        crs=ccrs.GOOGLE_MERCATOR,
        vdims=[metric, 'primary_location_id']  # Include location_id as dimension
    ).opts(
        tools=["tap", "hover"],
        color=metric,
        size=10,
        width=800,
        height=500,
        selection_alpha=1,
        nonselection_alpha=0.4,
        selection_color="red",
    )
    
    # Attach a selection stream
    selection = hv.streams.Selection1D(source=points)
    
    # Register the callback
    selection.add_subscriber(selection_callback)
    
    return (tiles * points) #.opts(width=600, height=500)

In [11]:
def selected_location_metrics_table(location_id):
    # Show metrics for selected location by configuration_name, transposed for easier viewing
    if not location_id:
        return pn.pane.Markdown("Select a location to view its metrics.")

    filtered_gdf = state.metrics_gdf[state.metrics_gdf['primary_location_id'] == location_id]
    if filtered_gdf.empty:
        return pn.pane.Markdown(f"No metrics found for location ID: {location_id}")
    
    filtered_gdf = filtered_gdf.drop(columns=['geometry'])
    # print(filter_gdf)

    # Transpose for easier viewing
    transposed_gdf = filtered_gdf.set_index(['configuration_name', 'variable_name']).T.reset_index()

    table = pn.widgets.Tabulator(
        transposed_gdf,
        theme="bootstrap",
        pagination='remote',
        page_size=10,
        sizing_mode='stretch_width',
    )
    
    return table


In [ ]:
def get_location_details_title(
    locations_gdf: gpd.GeoDataFrame,
    location_id: str,
    loading: bool = False
):
    if not location_id:
        return pn.pane.Markdown("# Location Details Pane")
    
    # Filter to find the specific location
    filtered_gdf = locations_gdf[locations_gdf["id"] == location_id]
    
    # Get location name if available
    location_name = ""
    if not filtered_gdf.empty and "name" in filtered_gdf.columns:
        location_name = filtered_gdf["name"].iloc[0]
    
    # Create title with location ID and name on same line
    if location_name:
        title = f"# Location Details for {location_id} - {location_name}"
    else:
        title = f"# Location Details for {location_id}"
        
    if not loading:
        return pn.pane.Markdown(title)
    
    return pn.Column(
        pn.pane.Markdown(title),
        pn.indicators.LoadingSpinner(value=True, size=40, name='Loading...')
    )


bound_location_details_title = pn.bind(
    get_location_details_title,
    locations_gdf=state.param.locations_gdf,
    location_id=state.param.location_id,
    loading=state.param.timeseries_loading
)

In [15]:
locations_widget = pn.widgets.Select(
    name="location",
    options=list(state.metrics_gdf['primary_location_id']),
    value=state.location_id,
)
metrics_widget = pn.widgets.Select(
    name="metric",
    options=list(state.metrics),
    value=state.metrics[0],
)
configurations_widget = pn.widgets.Select(
    name="configuration",
    options=list(state.configuration_names),
    value=state.configuration_names[0],
)
variables_widget = pn.widgets.Select(
    name="variable",
    options=list(state.variable_names),
    value=state.variable_names[0],
)

# Calculate default date range: 30 days before and after current date (UTC)
# current_date = datetime.now(datetime.timezone.utc)
# default_start_date = current_date - timedelta(days=30)
# default_end_date = current_date + timedelta(days=30)
default_start_date = datetime(2018, 1, 1)
default_end_date = datetime(2020, 1, 1)

# Date filter widgets - using DatetimePicker for better UI
start_date_widget = pn.widgets.DatetimePicker(
    name="Start Date (UTC)",
    value=default_start_date,
    enable_time=True,
    enable_seconds=False,
    military_time=False
)
end_date_widget = pn.widgets.DatetimePicker(
    name="End Date (UTC)", 
    value=default_end_date,
    enable_time=True,
    enable_seconds=False,
    military_time=False
)

# Note: load_timeseries_button is defined in an earlier cell

bound_locations_map = pn.bind(
    get_locations_map,
    metrics_gdf=state.param.metrics_gdf,
    configuration_name=configurations_widget.param.value,
    variable_name=variables_widget.param.value,
    metric=metrics_widget.param.value
)

tabulator = pn.widgets.Tabulator(
    metrics_gdf.drop(columns=["geometry"]), 
    theme="bootstrap", 
    pagination="remote", 
    page_size=10
)

# Updated bound timeseries plot with button trigger and date filtering
bound_timeseries_plot = pn.bind(
    get_timeseries_plot,
    location_id=state.param.location_id,
    load_trigger=load_timeseries_button.param.clicks,
    start_date=start_date_widget.param.value,
    end_date=end_date_widget.param.value
)

bound_selected_location_metrics_table = pn.bind(
    selected_location_metrics_table,
    location_id=state.param.location_id
)

top_pane = pn.Tabs(
    ("Location Metrics Map", pn.Column(pn.Row(configurations_widget,variables_widget,metrics_widget),pn.Row(bound_locations_map))),
    ("Location Metrics List", tabulator)
)

sidebar = pn.Column(
    pn.pane.Markdown(f"""
        ### 🔧 Database Connection Information
        ```
        User: {utils.TRINO_USER}
        Catalog: {utils.TRINO_CATALOG}
        Schema: {utils.TRINO_SCHEMA}
        ```
    """),
    "---",
    pn.pane.Markdown("### 📅 Date Filters"),
    start_date_widget,
    end_date_widget,
    pn.pane.Markdown("*Leave blank for no date filtering*"),
    "---",
    pn.pane.Markdown("### 📊 Timeseries Loading"),
    load_timeseries_button,
    pn.pane.Markdown("*Select a location, then click to load timeseries*"),
    "---",
)

main = pn.Column(
    top_pane,
    bound_location_details_title,
    bound_selected_location_metrics_table,
    bound_timeseries_plot,
)

pn.template.MaterialTemplate(
    site="TEEHR",
    title="Simulation Dashboard",
    sidebar=[sidebar],
    main=[main],
).servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo(dark_mode=False, device_pixel_ratio=2)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-140332507187728] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Tabs(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                [0] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                    [0] Select(design=<class 'panel.theme.materi..., name='configuration', options=['nwm30_retrospective'], sizing_mode='stretch_width', value='nwm30_retrospective')
                    [1] Select(design=<class 'panel.theme.materi..., name='variable', options=['streamflow_hourly_inst',...], sizing_mode='stretch_width', value='streamflow_hourly_inst')
                    [2] Select(design=<class 'panel.theme.materi..., name='metric', options=['average', 'count', ...], sizing_mode='stretch_width', value='average')
                [1] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [1] Tabulator(design=<class 'panel.theme.materi..., page_size=10, pagination='remote', sizing_mode='stretch_width', theme='bootstrap', value=     primary_location_id  ...)
        [1] ParamFunction(function, _pane=Markdown, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [2] ParamFunction(function, _pane=Markdown, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [3] ParamFunction(function, _pane=Markdown, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-140332507195312] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [1] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [2] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [3] DatetimePicker(as_numpy_datetime64=False, design=<class 'panel.theme.materi..., enable_seconds=False, military_time=False, name='Start Date (UTC)', sizing_mode='stretch_width', value=datetime.datetime(2018, ...)
        [4] DatetimePicker(as_numpy_datetime64=False, design=<class 'panel.theme.materi..., enable_seconds=False, military_time=False, name='End Date (UTC)', sizing_mode='stretch_width', value=datetime.datetime(2020, ...)
        [5] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [6] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [7] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [8] Button(button_type='primary', design=<class 'panel.theme.materi..., margin=(10, 5), name='Load Timeseries', width=150)
        [9] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [10] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')